In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../../../pycore/')
from simulate import RBA_result
from utils import extract_details_from_rxnid

import matplotlib.pyplot as plt
import matplotlib.patches as mp
import seaborn as sns
import matplotlib.lines as mlines
%matplotlib inline

In [2]:
path_enz_mw = '../../../input/enz_mw_g_per_mmol.txt'
path_pro_mw = '../../../input/pro_mw_g_per_mmol.txt'

with open(path_enz_mw) as f:
    text = f.read().split('\n')
text = [i for i in text if i != '']
enz_mw = dict()
for i in text:
    k,v = i.split('\t')
    enz_mw[k] = float(v)
    
with open(path_pro_mw) as f:
    text = f.read().split('\n')
text = [i for i in text if i != '']
pro_mw = dict()
for i in text:
    k,v = i.split('\t')
    pro_mw[k] = float(v)

In [3]:
with open('../../../input/RBA_kapp_batch_per_hour.txt') as f:
    x = f.read().split('\n')
x = [i for i in x if i not in ['', '/']]

kapps = dict()
for line in x:
    rxn,kapp = line.split(' ')
    kapps[rxn[1:-1]] = float(kapp)

In [4]:
flist = ['vglc_0_5', 'vglc_1_0', 'vglc_2_0', 'vglc_3_0', 'vglc_4_0', 'vglc_5_0',
         'vglc_5_5', 'vglc_5_6', 'vglc_5_7', 'vglc_5_8', 'vglc_5_9', 'vglc_6_0', 'vglc_7_0',
         'vglc_8_0', 'vglc_9_0', 'vglc_10_0', 'vglc_11_0', 'vglc_12_0',
         'vglc_13_0', 'vglc_14_0', 'vglc_15_0', 'vglc_16_0', 'vglc_17_0']
reslist = []
for fdir in flist:
    res = RBA_result(biom_id=None)
    res.load_from_json('./results_capacitycorrected/' + fdir + '/RBA_result.json')
    reslist.append(res)

In [5]:
rxns_check = ['ENZLOAD-DBTS_c_FWD-YNR057C', 'ENZLOAD-DBTS_c_REV-YNR057C',
              'ENZLOAD-CYCCL_m_FWD-YAL039C', 'ENZLOAD-CYCCL_m_REV-YAL039C',
              'ENZLOAD-DADK_c_FWD-YDL166C', 'ENZLOAD-DADK_c_REV-YDL166C',
              'ENZLOAD-BTDDH_c_FWD-YAL060W', 'ENZLOAD-BTDDH_c_REV-YAL060W']
spec_pairs = [('ENZLOAD-ACACT40ir_c_REV-YPL028W', 'ENZLOAD-ACACT40ir_c_FWD-YPL028W'),
              ('ENZLOAD-ACACT40ir_m_REV-YPL028W', 'ENZLOAD-ACACT40ir_m_FWD-YPL028W'),
              ('ENZLOAD-TPI_c_REV-YDR050C', 'ENZLOAD-TPI_c_FWD-YDR050C')]

In [6]:
N = len(flist)
for i in range(0,N):
    fname = flist[i]
    res = reslist[i]
    gr = res.growth_rate
    if 'EX_etoh_e' in res.metabolic_flux.keys():
        etoh = res.metabolic_flux['EX_etoh_e']
    else:
        etoh = 0
    print(fname, ', GR =', round(gr,4), ', ETOH =', round(etoh,4))
    enzloads = {k:v for k,v in res.raw_flux.items() \
                if k.split('-')[0] == 'ENZLOAD' and v > 0}
    eload_raw = res.proteome_capacity_usage
    
    load_calc_denominator = 0.55 * res.raw_flux['BIOSYN-PROTTOBIO']
    
    eload_excess = 0
    for spec_rev,spec_fwd in spec_pairs:
        if spec_rev in enzloads.keys():
            # Subtract exchange reverse load
            eload = res.raw_flux[spec_rev] * enz_mw[spec_rev]
            print(spec_rev, round(eload / load_calc_denominator,4))
            eload_excess += eload
            
            # Subtract exchange forward load
            eload = res.raw_flux[spec_rev] * enz_mw[spec_fwd] * \
                kapps['RXN-'+spec_rev[8:]] / kapps['RXN-'+spec_fwd[8:]]
            eload_excess += eload
            print(spec_fwd, round(eload / load_calc_denominator,4))
        
    for rxn in rxns_check:
        if rxn in enzloads.keys():
            eload = res.raw_flux[rxn] * enz_mw[rxn]
            print(rxn, round(eload / load_calc_denominator,4))
            eload_excess += eload
            
    eload_new = eload_raw - eload_excess / load_calc_denominator
    print('Pre-corrected capacity usage:', round(eload_raw, 4))
    print('Corrected capacity usage:', round(eload_new,4))

vglc_0_5 , GR = 0.0398 , ETOH = 0.0
Pre-corrected capacity usage: 0.0701
Corrected capacity usage: 0.0701
vglc_1_0 , GR = 0.0828 , ETOH = 0
Pre-corrected capacity usage: 0.1418
Corrected capacity usage: 0.1418
vglc_2_0 , GR = 0.1681 , ETOH = 0
Pre-corrected capacity usage: 0.2798
Corrected capacity usage: 0.2798
vglc_3_0 , GR = 0.2525 , ETOH = 0
Pre-corrected capacity usage: 0.4134
Corrected capacity usage: 0.4134
vglc_4_0 , GR = 0.336 , ETOH = 0
Pre-corrected capacity usage: 0.543
Corrected capacity usage: 0.543
vglc_5_0 , GR = 0.4188 , ETOH = 0
Pre-corrected capacity usage: 0.6688
Corrected capacity usage: 0.6688
vglc_5_5 , GR = 0.4593 , ETOH = 0
Pre-corrected capacity usage: 0.7232
Corrected capacity usage: 0.7232
vglc_5_6 , GR = 0.4662 , ETOH = 0
Pre-corrected capacity usage: 0.7245
Corrected capacity usage: 0.7245
vglc_5_7 , GR = 0.4675 , ETOH = 0.1599
Pre-corrected capacity usage: 0.7258
Corrected capacity usage: 0.7258
vglc_5_8 , GR = 0.4672 , ETOH = 0.3846
Pre-corrected capacit